# Bokeh dashboard for THOR data

In [1]:
import os, io, random
import string
import pickle
import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns
from collections import OrderedDict
import datetime as dt
import geopandas as gpd

In [19]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider, CustomJS, DatePicker
from bokeh.plotting import figure, show
from bokeh.themes import Theme
from bokeh.io import show, output_notebook
from bokeh.models import (DataTable, GeoJSONDataSource, ColumnDataSource, HoverTool, renderers,
                          Label, LabelSet, CustomJS, MultiSelect, Dropdown, Div)

output_notebook()
import panel as pn
import panel.widgets as pnw
pn.extension()

Loading BokehJS ...

In [60]:
s = pd.read_pickle('thor_small.pkl')
x = s[~s.TGTLATDD_DDD_WGS84.isnull()].copy()
countries = ['NORTH VIETNAM','SOUTH VIETNAM','LAOS','THAILAND','CAMBODIA']
x = x[x.TGTCOUNTRY.isin(countries)]
#x = x.drop(columns=['MSNDATE'])
#gdf=gpd.GeoDataFrame(s, geometry=gpd.points_from_xy(s.TGTLONDDD_DDD_WGS84, s.TGTLATDD_DDD_WGS84),crs="EPSG:4326").dropna()

In [8]:
colormap={'NORTH VIETNAM':'brown','SOUTH VIETNAM':'orange','LAOS':'red',
                'CAMBODIA':'green','THAILAND':'blue','UNKNOWN':'gray'}
x.MFUNC_DESC.unique()
import matplotlib
cmap = matplotlib.colormaps.get_cmap('Set1')
#names = cols
#olors = cmap(np.linspace(0, 1, len(names)))
#m={i:c for i, c in zip(names, colors)}

providers = ['CARTODBPOSITRON','STAMEN_TERRAIN','OSM','ESRI_IMAGERY']
cats = ['TGTCOUNTRY','WEAPONTYPE','MFUNC_DESC']

In [78]:
def draw_map(df=None, long=None, lat=None, height=600, colorby='TGTCOUNTRY',
             point_size=5,
              tile_provider="CartoDB Positron"):
  
    tools = "pan,wheel_zoom,box_zoom,hover,tap,lasso_select,reset,save"
    sizing_mode='stretch_both'

    # range bounds supplied in web mercator coordinates
    k = 6378137
    pad = 700000
    if lat == None:
        lat = 16
    if long == None:
        long = 108
    x = long * (k * np.pi/180.0)
    y = np.log(np.tan((90 + lat) * np.pi/360.0)) * k
      
    p = figure(x_range=(x-pad, x+pad), y_range=(y-pad, y+pad),
               x_axis_type="mercator", y_axis_type="mercator", tools=tools,
               width=height, height=height)
    p.add_tile(tile_provider)
    if df is None:
        return
    df.loc[:,'color'] = [colormap[i] if i in colormap else 'gray' for i in df[colorby]]
    #df['size'] = 10
    source = ColumnDataSource(df)    
    p.circle(x='x', y='y', size=point_size, alpha=0.7, color='color', source=source)#, legend_group=colorby)
    p.toolbar.logo = None    
    p.title.text = "date"
    '''hover = p.select(dict(type=HoverTool))
    hover.tooltips = OrderedDict([
        ("TGTCOUNTRY", "@TGTCOUNTRY"),
        ("MSNDATE", "@MSNDATE{%F}"),
        ("TAKEOFFLOCATION", "@TAKEOFFLOCATION"),
        ("WEAPONTYPE", "@WEAPONTYPE"),
        ("MFUNC_DESC", "@MFUNC_DESC")     
    ])
    hover.formatters={'@MSNDATE': 'datetime'}'''
    return p 

In [ ]:
#p=draw_map(x[:500])
#show(p)

## dashboard 

In [80]:
def dashboard():
    cols = list(x.columns)
    colorby='TGTCOUNTRY'
    map_pane=pn.pane.Bokeh(width=700)
    df_pane = pn.pane.DataFrame(width=600,height=600)
    date_picker = pnw.DatePicker(name='Pick Date',width=200)
    from datetime import date  
    date_picker.value=date(1965, 1, 1)    
    date_slider = pnw.DateSlider(name='Date', start=dt.datetime(1965, 1, 1), 
                                 end=dt.datetime(1973, 10, 31), value=dt.datetime(1968, 1, 1))      
    tile_select = pnw.Select(name='tile layer',options=providers,width=200)
    filterby_select = pnw.Select(name='filter by',value='',options=['']+cols[1:5],width=200)
    value_select = pnw.Select(name='value',value='',options=[],width=200)
    find_btn = pnw.Button(name='find in region',button_type='primary',width=200)

    def update_tile(event=None):
        p = map_pane.object
        p.renderers = [x for x in p.renderers if not str(x).startswith('TileRenderer')]
        rend = renderers.TileRenderer(tile_source= get_provider(tile_select.value))
        p.renderers.insert(0, rend)

    def update_filter(event):
        col=filterby_select.value
        if col=='':
            value_select.options = []
        else:
            value_select.options = sorted(list(x[col].dropna().unique()))

    def find_in_region(event):
        #get points in selected map area
        p = map_pane.object
        source = p.renderers[1].data_source
        d = x[(x.x>p.x_range.start) & (x.x<p.x_range.end) & (x.y>p.y_range.start) & (x.y<p.y_range.end)]
        #add any filter
        d = do_filter(d)
        if len(d)==0:
            return
        elif len(d)>150000:           
            p.title.text = 'too many points!'
        else: 
            d.loc[:,'color'] = [colormap[i] if i in colormap else 'gray' for i in d[colorby]]        
            source.data = dict(d)
            p.title.text = 'selected %s points' %len(d)
        map_pane.param.trigger('object')     
        return

    def extract_points():
        """save points"""

        
        return
        
    def do_filter(d):
        col = filterby_select.value
        val = value_select.value
        if col != '':
            d = d[d[col]==val].copy()
        return d
    
    def update_date(event):
        date_slider.value = date_picker.value     
        
    def update_map(event=None, date=None):
        p = map_pane.object
        source = p.renderers[1].data_source
        if date == None:
            date = str(date_slider.value)
        d = x[x.MSNDATE==date]
        d = do_filter(d)
        if len(d)==0:
            return  
        d.loc[:,'color'] = [colormap[i] if i in colormap else 'gray' for i in d[colorby]]
        source.data = dict(d)
        p.title.text = date
        
    sdate='1968-01-01'
    d = x[x.MSNDATE==sdate].copy()
    map_pane.object=draw_map(d)
    
    date_slider.param.watch(update_map,'value')
    date_picker.param.watch(update_date,'value')
    tile_select.param.watch(update_tile,'value')
    filterby_select.param.watch(update_filter,'value')
    value_select.param.watch(update_map,'value')
    find_btn.on_click(find_in_region)
    
    dashboard = pn.Column(date_slider,pn.Row(
                pn.Column(date_picker,tile_select,filterby_select,
                value_select,find_btn),map_pane))
    return dashboard

app=dashboard()

In [81]:
app

/tmp/ipykernel_819353/1822654874.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.loc[:,'color'] = [colormap[i] if i in colormap else 'gray' for i in d[colorby]]


Column
    [0] DateSlider(end=datetime.datetime(1973, ..., name='Date', start=datetime.datetime(1965, ..., value=datetime.datetime(1968, ...)
    [1] Row
        [0] Column
            [0] DatePicker(name='Pick Date', value=datetime.date(1965, 1, 1), width=200)
            [1] Select(name='tile layer', options=['CARTODBPOSITRON', ...], value='CARTODBPOSITRON', width=200)
            [2] Select(name='filter by', options=['', 'TGTCOUNTRY', ...], width=200)
            [3] Select(name='value', width=200)
            [4] Button(button_type='primary', name='find in region', width=200)
        [1] Bokeh(figure, align='auto', height=600, width=700)

In [67]:
x

,MSNDATE,TGTCOUNTRY,TAKEOFFLOCATION,WEAPONTYPE,MFUNC_DESC,TGTLATDD_DDD_WGS84,TGTLONDDD_DDD_WGS84,YEAR,MONTH,MONTHNAME,x,y
485519,1965-10-01,NORTH VIETNAM,TAKHLI,NaN,ARMED RECCE,17.800000,106.233333,1965,10,October,1.182584e+07,2.014152e+06
2959062,1965-10-01,NORTH VIETNAM,CORAL SEA (CVA-43),LAU-3A/AERO7,ARMED RECCE,18.500000,105.350000,1965,10,October,1.172751e+07,2.096157e+06
469853,1965-10-01,SOUTH VIETNAM,DANANG,NaN,AIR INTERDICTION,14.551360,108.633881,1965,10,October,1.209307e+07,1.637550e+06
2959008,1965-10-01,SOUTH VIETNAM,BIEN HOA,LAU-3A/AERO7,STRIKE,10.788045,106.401189,1965,10,October,1.184453e+07,1.208079e+06
2893165,1965-10-01,NORTH VIETNAM,KORAT,750LB GP M-117,STRIKE,18.333333,105.750000,1965,10,October,1.177204e+07,2.076602e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
117242,1975-05-15,CAMBODIA,NaN,NaN,AIR INTERDICTION,10.627222,103.628611,1975,5,May,1.153588e+07,1.189859e+06
4125052,1975-05-15,CAMBODIA,NaN,MK-20 AN TANK/MTL,AIR INTERDICTION,10.500555,103.638333,1975,5,May,1.153697e+07,1.175515e+06
4597273,1975-05-15,CAMBODIA,NaN,NaN,AIR INTERDICTION,10.627222,103.628611,1975,5,May,1.153588e+07,1.189859e+06
56543,1975-05-15,CAMBODIA,NaN,NaN,AIR INTERDICTION,10.500555,103.638333,1975,5,May,1.153697e+07,1.175515e+06
